In [10]:
import os
import pandas as pd
import pyspark as pa
from uuid import uuid4 as uidv4

In [3]:
file_path = './data/house.csv'
house = pd.read_csv(file_path)
house.shape

(23677, 12)

In [4]:
house.columns

Index(['市区', '小区', '户型', '朝向', '楼层', '装修情况', '电梯', '面积(㎡)', '价格(万元)', '年份',
       'Id', 'Unnamed: 11'],
      dtype='object')

In [5]:
house.head(3)

,市区,小区,户型,朝向,楼层,装修情况,电梯,面积(㎡),价格(万元),年份,Id,Unnamed: 11
0,朝阳,育慧里一区,1室0厅,西,7,精装,有电梯,52,343,2001,1.01E+11,NaN
1,朝阳,大西洋新城A区,2室2厅,南北,10,精装,有电梯,86,835,1999,1.01E+11,NaN
2,朝阳,团结湖路,2室1厅,东西,6,精装,无电梯,65,430,1980,1.01E+11,NaN


In [6]:
house = house.drop(columns='Unnamed: 11')
house.head(3)

,市区,小区,户型,朝向,楼层,装修情况,电梯,面积(㎡),价格(万元),年份,Id
0,朝阳,育慧里一区,1室0厅,西,7,精装,有电梯,52,343,2001,1.01E+11
1,朝阳,大西洋新城A区,2室2厅,南北,10,精装,有电梯,86,835,1999,1.01E+11
2,朝阳,团结湖路,2室1厅,东西,6,精装,无电梯,65,430,1980,1.01E+11


In [11]:
house["Id"] = house["Id"].apply(lambda _: uidv4())
house.head(3)

,市区,小区,户型,朝向,楼层,装修情况,电梯,面积(㎡),价格(万元),年份,Id
0,朝阳,育慧里一区,1室0厅,西,7,精装,有电梯,52,343,2001,bc913ac6-b0c2-4dab-85e5-108df77a6e93
1,朝阳,大西洋新城A区,2室2厅,南北,10,精装,有电梯,86,835,1999,e1bb4a17-df27-4452-b427-034f69a995e5
2,朝阳,团结湖路,2室1厅,东西,6,精装,无电梯,65,430,1980,caa1c5f7-58e3-4b45-acf3-708a61842203


In [14]:
house.dtypes

市区        object
小区        object
户型        object
朝向        object
楼层        object
装修情况      object
电梯        object
面积(㎡)     object
价格(万元)    object
年份        object
Id        object
dtype: object

In [18]:
# filter out invalid rows
house.value_counts("楼层")

楼层
6       7656
18      1553
5       1070
16      1033
24       858
7        821
11       790
15       785
14       745
12       702
9        669
21       644
20       638
22       577
28       505
4        465
26       456
10       406
13       405
27       402
17       373
25       357
19       347
8        321
23       253
3        201
29       166
30       126
32        99
2         94
31        64
33        33
34        21
35        17
36         8
1          6
40         3
57         1
490        1
42         1
京师吉地       1
房山         1
有电梯        1
朝阳         1
精装         1
Name: count, dtype: int64

In [ ]:
def is_floor_valid(floor, top) -> bool:
    try:
        floor = int(floor)
        return floor > 0 and floor < top 
    except ValueError:
        return False

house_filtered = house.apply(lam)

In [15]:
house_type = {
    '市区': str,
    '小区': str,
    '户型': str,
    '朝向': str,
    '楼层': int,
    '装修情况': str,
    '电梯': str,
    '面积(㎡)': float,
    '价格(万元)': float,
    '年份': str,
    'Id': str,
}
house.astype(house_type)
house.dtypes

ValueError: invalid literal for int() with base 10: '精装': Error while type casting for column '楼层'

In [19]:
def foo(s: str):
    s = int(s)
    print(type(s))

k = "1"
foo(k)
print(type(k))

<class 'int'>
<class 'str'>


In [20]:
k = "aaa"
int(k)

ValueError: invalid literal for int() with base 10: 'aaa'